# PostgreSQL Fixes   

> Like any DBMS, PostgreSQL requires careful administration. Our implementation is a little quick-and-dirty, so here, we include functions to fix errors as they arise.

## Basics

### Python Imports

Setting python imports, environment variables, and other crucial set up parameters here.  

In [2]:
from alhazen.aliases import *
from alhazen.core import lookup_chat_models
from alhazen.agent import AlhazenAgent
from alhazen.schema_sqla import *
from alhazen.core import lookup_chat_models
from alhazen.tools.basic import AddCollectionFromEPMCTool, DeleteCollectionTool
from alhazen.tools.paperqa_emulation_tool import PaperQAEmulationTool
from alhazen.tools.metadata_extraction_tool import * 
from alhazen.tools.protocol_extraction_tool import *
from alhazen.tools.tiab_classifier_tool import *
from alhazen.tools.tiab_extraction_tool import *
from alhazen.tools.tiab_mapping_tool import *
from alhazen.toolkit import *
from alhazen.utils.ceifns_db import Ceifns_LiteratureDb, create_ceifns_database, drop_ceifns_database, backup_ceifns_database, list_databases

from alhazen.utils.searchEngineUtils import *


from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.pgvector import PGVector
from langchain_community.chat_models.ollama import ChatOllama
from langchain_google_vertexai import ChatVertexAI
from langchain_openai import ChatOpenAI

from bs4 import BeautifulSoup,Tag,Comment,NavigableString
from databricks import sql
from datetime import datetime
from importlib_resources import files
import os
import pandas as pd
from pathlib import Path
import re
import requests

from sqlalchemy import text, create_engine, exists, func, or_, and_, not_, desc, asc
from sqlalchemy.orm import sessionmaker, aliased

from time import time,sleep
from tqdm import tqdm
from urllib.request import urlopen
from urllib.parse import quote_plus, quote, unquote
from urllib.error import URLError, HTTPError
import uuid
import yaml

### Environment Variables

Remember to set environmental variables for this code:

* `ALHAZEN_DB_NAME` - the name of the PostGresQL database you are storing information into
* `LOCAL_FILE_PATH` - the location on disk where you save temporary files, downloaded models or other data.   

In [3]:
if os.environ.get('LOCAL_FILE_PATH') is None: 
    raise Exception('Where are you storing your local literature database?')
if os.path.exists(os.environ['LOCAL_FILE_PATH']) is False:
    os.makedirs(os.environ['LOCAL_FILE_PATH'])  
loc = os.environ.get('LOCAL_FILE_PATH')

## Backup all databases


In [12]:
loc = os.environ['LOCAL_FILE_PATH']
current_date_time = datetime.now()
formatted_date_time = f'{current_date_time:%Y-%m-%d-%H-%M-%S}'
formatted_date = f'{current_date_time:%Y-%m-%d}'
if os.path.exists(loc+'/backups') is False:
    os.makedirs(loc+'/backups')
for db in list_databases():
    print('Backing up database:',db)
    backup_path = loc+'/backups/'+db+'_backup_'+formatted_date_time+'.sql'
    backup_ceifns_database(db, backup_path)
    print('~~~~~~~~~~~~~~~~~~~~~~~~~')


### Try some queries

In [4]:
db_name = 'em_tech'
ldb = Ceifns_LiteratureDb(loc=loc, name=db_name)

In [14]:
# search for papers using embeddings
q = ldb.session.query(SKE.id) \
        .distinct() \
        .filter(SKC.id==SKC_HM.ScientificKnowledgeCollection_id) \
        .filter(SKC_HM.has_members_id==SKE.id) \
        .filter(SKE.id==SKE_HR.ScientificKnowledgeExpression_id) \
        .filter(SKE_HR.has_representation_id==SKI.id) \
        .filter(SKC.id == '3') \
        .filter(or_(SKI.type == 'JATSFullText', SKI.type == 'PDFFullText')) 
for e_id in q.all():
    print(e_id)

('doi:10.1002/1873-3468.13916',)
('doi:10.1007/s10974-017-9477-5',)
('doi:10.1016/j.cell.2021.01.033',)
('doi:10.1016/j.celrep.2020.02.003',)
('doi:10.1016/j.celrep.2023.112107',)
('doi:10.1016/j.celrep.2023.112432',)
('doi:10.1016/j.devcel.2019.09.019',)
('doi:10.1016/j.jmb.2021.167423',)
('doi:10.1016/j.jsb.2017.07.007',)
('doi:10.1016/j.jsb.2018.02.001',)
('doi:10.1016/j.jsb.2019.08.006',)
('doi:10.1016/j.jsb.2022.107911',)
('doi:10.1016/j.str.2018.03.015',)
('doi:10.1016/j.str.2020.07.018',)
('doi:10.1016/j.str.2020.12.014',)
('doi:10.1016/j.str.2021.10.012',)
('doi:10.1038/nature26003',)
('doi:10.1038/s41467-020-14350-9',)
('doi:10.1038/s41467-020-14535-2',)
('doi:10.1038/s41467-020-17466-0',)
('doi:10.1038/s41467-020-18777-y',)
('doi:10.1038/s41467-020-18952-1',)
('doi:10.1038/s41467-021-24887-y',)
('doi:10.1038/s41467-022-29322-4',)
('doi:10.1038/s41467-022-32584-7',)
('doi:10.1038/s41467-022-33221-z',)
('doi:10.1038/s41467-022-35409-9',)
('doi:10.1038/s41467-023-36372-9',)
('do

In [16]:
c_ids = ['2']
collections_clause = ' OR '.join(['skc_hm."ScientificKnowledgeCollection_id"=\'{}\''.format(coll_id) for coll_id in c_ids])
ldb.session.rollback()
q2_all = """
    SELECT DISTINCT ske.id, ske.content, ske.publication_date as pub_date, ske.type as pub_type, emb.embedding, skf.content 
    FROM langchain_pg_embedding as emb, 
        "ScientificKnowledgeExpression" as ske,
        "ScientificKnowledgeCollection_has_members" as skc_hm, 
        "ScientificKnowledgeFragment" as skf
    WHERE skc_hm.has_members_id = ske.id AND 
        emb.cmetadata->>'i_type' = 'CitationRecord' AND
        emb.cmetadata->>'e_id' = ske.id AND 
        emb.cmetadata->>'f_id' = skf.id AND ({})
    ORDER BY pub_date DESC;
    """.format(collections_clause)

ldb.session.execute(text(q2_all)).fetchall()

[('doi:10.1101/2024.04.04.586917', 'Last MG, Abendstein L, Voortman LM, Sharp TH. (2024) Ais: streamlining segmentation of cryo-electron tomography datasets', datetime.date(2024, 4, 4), 'ScientificPrimaryResearchPreprint', '[0.01440517,0.00869043,-0.016746111,0.013716584,0.00030222267,-0.0056080488,-0.016920777,0.007332475,0.0037339602,0.052169465,0.016712014,-0.02946986 ... (12405 characters truncated) ... -0.041706212,0.043343987,0.06610469,-0.018040305,-0.044764996,-0.019441947,0.027401354,0.03635239,0.029366896,-0.02507872,-0.04119841,-0.00039106846]', 'Ais: streamlining segmentation of cryo-electron tomography datasets'),
 ('doi:10.1371/journal.ppat.1011750', 'de Sautu M, Herrmann T, Scanavachi G, Jenni S, Harrison SC. (2024) The rotavirus VP5*/VP8* conformational transition permeabilizes membranes to Ca2.', datetime.date(2024, 4, 4), 'ScientificPrimaryResearchArticle', '[-0.012906601,0.00039328026,0.0054566865,0.019395705,-0.015244957,0.013945329,-0.032279227,0.019900467,0.004740

#### Add a collection based on EMPIAR papers

In [ ]:
addEMPCCollection_tool = [t for t in cb.tk.get_tools() if isinstance(t, AddCollectionFromEPMCTool)][0]
step = 20
for start_i in range(0, len(empiar_dois), step):
    query = ' OR '.join(['doi:"'+empiar_dois[i]+'"' for i in range(start_i, start_i+step)])
    addEMPCCollection_tool.run({'id': '3', 'name':'EMPIAR Papers', 'query':query, 'full_text':True})

In [ ]:
def join_set(x):
    out = ''
    try:
        out = ' '.join(set(x))
    except:
        pass
    return out

# identify papers that we have full text for in EMPIAR
q = ldb.session.query(SKE.id) \
        .distinct() \
        .filter(SKC.id==SKC_HM.ScientificKnowledgeCollection_id) \
        .filter(SKC_HM.has_members_id==SKE.id) \
        .filter(SKE.id==SKE_HR.ScientificKnowledgeExpression_id) \
        .filter(SKE_HR.has_representation_id==SKI.id) \
        .filter(SKC.id == '3') \
        .filter(or_(SKI.type == 'JATSFullText', SKI.type == 'PDFFullText')) 
dois_to_include = [d[0][4:] for d in q.all()]    

empiar_gold_standard = []
for i, row in empiar_df.iterrows():
    if row.doi in dois_to_include:
        empiar_gold_standard.append( row.to_dict() )
empiar_gold_standard_df = pd.DataFrame(empiar_gold_standard)

empiar_gs_df = empiar_gold_standard_df.groupby(['doi']).agg({'sample_preparation_type': join_set, 'agg_state': join_set, 'sample_preparation_buffer_ph': join_set, 
                                              'grid_model': join_set, 'grid_material': join_set, 'grid_mesh': join_set, 
                                              'grid_support_topology': join_set, 'grid_pretreatment': join_set, 'grid_vitrification_cryogen': join_set, 
                                              'grid_vit_ctemp': join_set, 'grid_vit_chumid': join_set}).reset_index()
empiar_gs_df

#### Import papers from DOIs pertaining to CryoET-Portal records `10000-10010`

The [CryoET Data portal](https://chanzuckerberg.github.io/cryoet-data-portal/python-api.html) system is based on submitted data to our curation team, accompanied by papers referenced by DOIs. Each dataset is assigned an ID value associated with DOIs. 

In [ ]:
# use the EMPCSearchTool to run a query for the dois mentioned
query = ' OR '.join(['doi:"'+d+'"' for d_id in dois for d in dois[d_id] ])
addEMPCCollection_tool = [t for t in cb.tk.get_tools() if isinstance(t, AddCollectionFromEPMCTool)][0]
addEMPCCollection_tool.run(tool_input={'id': '0', 'name':'CryoET Portal (10000-10010)', 'query':query, 'full_text':True})

#### Extend Database to include all CryoET papers

In [ ]:
import local_resources.queries.em_tech as em_tech_queries
from alhazen.utils.queryTranslator import QueryTranslator, QueryType

cols_to_include = ['ID', 'CORPUS_NAME', 'QUERY']
df = pd.read_csv(files(em_tech_queries).joinpath('EM_Methods.tsv'), sep='\t')
df = df.drop(columns=[c for c in df.columns if c not in cols_to_include])
df

In [ ]:
qt = QueryTranslator(df.sort_values('ID'), 'ID', 'QUERY', 'CORPUS_NAME')
(corpus_ids, epmc_queries) = qt.generate_queries(QueryType.epmc, sections=['TITLE_ABS', 'METHODS'])
corpus_names = df['CORPUS_NAME']

addEMPCCollection_tool = [t for t in cb.tk.get_tools() if isinstance(t, AddCollectionFromEPMCTool)][0]
for (id, name, query) in zip(corpus_ids, corpus_names, epmc_queries):
    if id != 2:
        continue
    addEMPCCollection_tool.run(tool_input={'id': id, 'name':name, 'query':query, 'full_text':False})

#### Combine + Sample CryoET + EMPIAR Collections to provide a test set of papers.

In [ ]:
ldb.create_new_collection_from_intersection('4', 'EMPIAR CryoET Papers', '2', '3')

#### Adding Machine Learning

In [ ]:
ml_query = '''
("Cryoelectron Tomography" OR "Cryo Electron Tomography" OR "Cryo-Electron Tomography" OR
    "Cryo-ET" OR "CryoET" OR "Cryoelectron Tomography" OR "cryo electron tomography" or 
    "cryo-electron tomography" OR "cryo-et" OR cryoet ) AND 
("Machine Learning" OR "Artificial Intelligence" OR "Deep Learning" OR "Neural Networks")
'''
addEMPCCollection_tool = [t for t in cb.tk.get_tools() if isinstance(t, AddCollectionFromEPMCTool)][0]
addEMPCCollection_tool.run(tool_input={'id': '6', 
                                       'name': 'Machine Learning in CryoET', 
                                       'query': ml_query, 
                                       'full_text': False})

In [ ]:
delCollection_tool = [t for t in cb.tk.get_tools() if isinstance(t, DeleteCollectionTool)][0]
delCollection_tool.run(tool_input={'collection_id': '6'})

#### Break up TIAB of papers into sentences + classify by discourse

NOTE - HUGGING FACE MODELS DO NOT WORK WELL ON THIS CORPUS. 
(NOT SURPRISINGLY - THEY WERE TRAINED ON MEDICAL PAPERS WHERE THE DIFFERENT SECTIONS OF THE PAPER WERE EXPLICITLY LABELED)

USE LLMS TO DO THE EXTRACTION - GPT3.5?

In [ ]:
# Get the metadata extraction tool
t2 = [t for t in cb.tk.get_tools() if isinstance(t, TitleAbstractDiscourseMappingTool)][0]
t2.run(tool_input={'collection_id': '5', 'run_label': 'dev'})


In [ ]:
j = '''{
"Background": "Eps15-homology domain containing proteins (EHDs) are eukaryotic, dynamin-related ATPases involved in cellular membrane trafficking. They oligomerize on membranes into filaments that induce membrane tubulation. While EHD crystal structures in open and closed conformations were previously reported, little structural information is available for the membrane-bound oligomeric form. Consequently, mechanistic insights into the membrane remodeling mechanism have remained sparse.",
"Objectives_Methods": "Here, by using cryo-electron tomography and subtomogram averaging, we determined structures of nucleotide-bound EHD4 filaments on membrane tubes of various diameters at an average resolution of 7.6 Å.",
"Results_Conclusions": "Assembly of EHD4 is mediated via interfaces in the G-domain and the helical domain. The oligomerized EHD4 structure resembles the closed conformation, where the tips of the helical domains protrude into the membrane. The variation in filament geometry and tube radius suggests a spontaneous filament curvature of approximately 1/70 nm<sup>-1</sup>. Combining the available structural and functional data, we suggest a model for EHD-mediated membrane remodeling."
}'''

json.loads(j)


In [ ]:

# Get the metadata extraction tool
models = ['databricks_dbrx']
for m in models:
    llm = llms_lookup.get(m)
    cb = AlhazenAgent(llm, llm, db_name=db_name)
    t2 = [t for t in cb.tk.get_tools() if isinstance(t, TitleAbstractDiscourseMappingTool)][0]
    t2.run(tool_input={'collection_id': '2', 'run_label': m, 'repeat_run': False})

In [ ]:
to_remove = ["doi:10.1101/2024.03.04.583254", 
             "doi:10.1101/2023.11.21.567712",
"doi:10.3791/6515",
"doi:10.1101/2023.07.28.550950",
"doi:10.1093/micmic/ozad067.483",
"doi:10.1007/978-1-0716-2639-9_20",
"doi:10.1016/j.yjsbx.2022.100076",
"doi:10.1016/j.xpro.2022.101658",
"doi:10.1016/j.cell.2022.06.034",
"doi:10.1093/plphys/kiab449",
"doi:10.1073/pnas.2118020118",
"doi:10.3791/62886",
"doi:10.20944/preprints202105.0098.v1",
"doi:10.1016/bs.mcb.2020.12.009",
"doi:10.1007/978-1-0716-0966-8_1",
"doi:10.1007/978-1-0716-0966-8_2",
"doi:10.21769/bioprotoc.3768",
"doi:10.1371/journal.ppat.1008883",
"doi:10.1101/2020.05.19.104828",
"doi:10.1073/pnas.1916331116",
"doi:10.1042/bst20170351_cor",
"doi:10.1038/s41594-018-0043-7",
"doi:10.1007/978-1-4939-8585-2_4",
"doi:10.1007/s41048-017-0040-0",
"doi:10.1007/978-1-4939-6927-2_20",
"doi:10.1016/j.str.2015.03.008",
"doi:10.1007/978-1-62703-227-8_4",
"doi:10.1016/b978-0-12-397945-2.00017-2",
"doi:10.1016/j.jmb.2010.10.021",
"doi:10.1186/1757-5036-3-6",
"doi:10.1016/j.jmb.2008.03.014",
"doi:10.1007/978-1-59745-294-6_20"]


for d in to_remove:
    q = """
    SELECT DISTINCT n.id FROM langchain_pg_embedding as emb, "Note" as n
    WHERE emb.cmetadata->>'n_type' = 'TiAbMappingNote__Discourse' AND
        emb.cmetadata->>'about_id' = '{}' AND 
        emb.cmetadata->>'discourse_type' = 'ResultsConclusions' AND 
        emb.cmetadata->>'n_id' = n.id;""".format(d)
    for row in ldb.session.execute(text(q)).all():
        ldb.delete_note(row[0], commit_this=True)
        

In [ ]:
ldb.session.rollback()
exp_q = ldb.session.query(SKE) \
        .filter(SKC_HM.has_members_id == SKE.id) \
        .filter(SKC_HM.ScientificKnowledgeCollection_id == str('2')) \
        .filter(SKE.id == SKE_HR.ScientificKnowledgeExpression_id) \
        .filter(SKE_HR.has_representation_id == SKI.id) \
        .filter(SKI.type=='CitationRecord') \
        .filter(or_(SKE.type == 'ScientificPrimaryResearchArticle', SKE.type == 'ScientificPrimaryResearchPreprint')) \
        .order_by(desc(SKE.publication_date))

count = 0
for e in tqdm(exp_q.all()):
    q = ldb.session.query(N) \
        .filter(N.id == NIA.Note_id) \
        .filter(NIA.is_about_id == e.id) \
        .filter(N.type =='TiAbMappingNote__Discourse')
    for n in q.all():
        dmap = json.loads(n.content) 
        if 'Objectives_Methods' in dmap.keys():
            print('beep')
            #new_dmap = {'Background': dmap.get('Background'), 'ObjectivesMethods': dmap.get('Objectives_Methods'), 'ResultsConclusions': dmap.get('Results_Conclusions')}
            #n.content = json.dumps(new_dmap, indent=4)
            #ldb.session.flush()
#ldb.session.commit()

In [ ]:
ldb.session.rollback()
exp_q = ldb.session.query(SKE) \
        .filter(SKC_HM.has_members_id == SKE.id) \
        .filter(SKC_HM.ScientificKnowledgeCollection_id == str('2')) \
        .filter(SKE.id == SKE_HR.ScientificKnowledgeExpression_id) \
        .filter(SKE_HR.has_representation_id == SKI.id) \
        .filter(SKI.type=='CitationRecord') \
        .filter(or_(SKE.type == 'ScientificPrimaryResearchArticle', SKE.type == 'ScientificPrimaryResearchPreprint')) \
        .order_by(desc(SKE.publication_date))

texts = []
metadatas = []

count = 0
for e in tqdm(exp_q.all()):
    q = ldb.session.query(N) \
        .filter(N.id == NIA.Note_id) \
        .filter(NIA.is_about_id == e.id) \
        .filter(N.type =='TiAbMappingNote__Discourse')


    n = q.first()
    dmap = json.loads(n.content)
    '''Runs through the list of expressions, generates embeddings, and stores them in the database'''

    for dtype in ['Background', 'ObjectivesMethods', 'ResultsConclusions']:
        t = dmap.get(dtype)
        if t is None:
            continue
        texts.append(t)
        metadatas.append({'about_id': e.id, \
                        'about_type': 'ScientificKnowledgeExpression', \
                        'n_id': n.id, \
                        'n_type': 'TiAbMappingNote__Discourse', \
                        'discourse_type': dtype})

docs = []
for t,m in zip(texts, metadatas):
    docs.append(Document(page_content=t, metadata=m))
    
db = PGVector.from_documents(
    embedding=ldb.embed_model,
    documents=docs,
    collection_name="Note"
)

In [ ]:
from transformers import pipeline, AutoModel, AutoTokenizer
import torch

model_path = '/Users/gully.burns/Documents/2024H1/models/discourse_tagger'
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1", 
                                          truncation=True, 
                                          max_length=512)
labels = ['BACKGROUND', 'OBJECTIVE', 'METHODS', 'RESULTS', 'CONCLUSIONS']
lookup = {'LABEL_%d'%(i):l for i, l in enumerate(labels)}
model = AutoModel.from_pretrained(model_path)
model.eval()

classifier = pipeline("text-classification", 
                      model = model_path, 
                      tokenizer=tokenizer, 
                      truncation=True,
                      batch_size=8,
                      device='mps')

In [ ]:
self = ldb
collection_id = '2'

q1 = self.session.query(SKE, SKI) \
        .filter(SKC_HM.ScientificKnowledgeCollection_id == collection_id) \
        .filter(SKC_HM.has_members_id == SKE.id) \
        .filter(SKE.id == SKE_HR.ScientificKnowledgeExpression_id) \
        .filter(SKE_HR.has_representation_id == SKI.id) \
        .filter(SKI.id == SKI_HP.ScientificKnowledgeItem_id) \
        .filter(SKI_HP.has_part_id == SKF.id) \
        .filter(SKI.type == 'CitationRecord') \
        .filter(or_(SKE.type == 'ScientificPrimaryResearchArticle', SKE.type == 'ScientificPrimaryResearchPreprint')) 

for ske, ski in tqdm(q1.all()):
    b = ''
    om = ''
    rc = ''  

    fragments = []
    for f in ski.has_part:
      if f.type in ['title', 'abstract']:
        fragments.append(f)

    # USE AN LLM HERE INSTEAD OF A DEEP LEARNING CLASSIFER


    for skf in sorted(fragments, key=lambda f: f.offset):
        for s in self.sent_detector.tokenize(skf.content):
            m = classifier(skf.content)
            l = lookup.get(m[0].get('label'))
            if l == 'BACKGROUND':
                if len(b) > 0:
                    b += '\n'
                b += s
            elif l == 'OBJECTIVE' or l == 'METHODS':
                if len(om) > 0:
                    om += '\n'
                om += s
            else: 
                if len(rc) > 0:
                    rc += '\n'
                rc += s
    skf_stem = ske.id+'.'+ski.type+'.'
    if len(b) > 0:
        f_b = ScientificKnowledgeFragment(id=str(uuid.uuid4().hex)[:10], 
                type='background_sentences', offset=-1, length=len(b),
                name=skf_stem+'background', content=b)
        self.session.add(f_b)
        ski.has_part.append(f_b)
        f_b.part_of = ski.id    
    if len(om) > 0:
        f_om = ScientificKnowledgeFragment(id=str(uuid.uuid4().hex)[:10], 
                type='objective_methods_sentences', offset=-1, length=len(om),
                name=skf_stem+'objective_methods', content=om)
        self.session.add(f_om)
        ski.has_part.append(f_om)
        f_om.part_of = ski.id
    if len(rc) > 0:
        f_rc = ScientificKnowledgeFragment(id=str(uuid.uuid4().hex)[:10], 
                type='results_conclusions_sentences', offset=-1, length=len(rc),
                name=skf_stem+'results_conclusions', content=rc)
        self.session.add(f_rc)
        ski.has_part.append(f_rc)
        f_rc.part_of = ski.id
    self.session.flush()
self.session.commit()

In [ ]:
self = ldb
collection_id = '2'
#self.session.rollback()
q2 = self.session.query(SKF) \
        .filter(SKC_HM.ScientificKnowledgeCollection_id == collection_id) \
        .filter(SKC_HM.has_members_id == SKE.id) \
        .filter(SKE.id == SKE_HR.ScientificKnowledgeExpression_id) \
        .filter(SKE_HR.has_representation_id == SKI.id) \
        .filter(SKI.id == SKI_HP.ScientificKnowledgeItem_id) \
        .filter(SKI_HP.has_part_id == SKF.id) \
        .filter(SKI.type == 'CitationRecord') \
        .filter(or_(SKF.type == 'results_conclusions_sentences', \
                SKF.type == 'objective_methods_sentences', \
                SKF.type == 'background_sentences'))
for skf in tqdm(q2.all()):
    self.delete_fragment(skf.id)
   


In [ ]:
self = ldb
collection_id = '2'
#self.session.rollback()
q2 = self.session.query(SKE, SKF) \
        .filter(SKC_HM.ScientificKnowledgeCollection_id == collection_id) \
        .filter(SKC_HM.has_members_id == SKE.id) \
        .filter(SKE.id == SKE_HR.ScientificKnowledgeExpression_id) \
        .filter(SKE_HR.has_representation_id == SKI.id) \
        .filter(SKI.id == SKI_HP.ScientificKnowledgeItem_id) \
        .filter(SKI_HP.has_part_id == SKF.id) \
        .filter(SKI.type == 'CitationRecord') \
        .filter(SKF.type == 'objective_methods_sentences') \
        .order_by(desc(SKE.publication_date)) \
        .order_by(SKF.name)

for ske, skf in tqdm(q2.all()):
    print(skf)


#### Get full text copies of all the papers about CryoET


In [ ]:
cb.agent_executor.invoke({'input':'Get full text copies of all papers in the collection with id="2".'})

In [ ]:
ldb.create_new_collection_from_sample('5', 'EMPIAR CryoET Papers Tests', '4', 20, ['ScientificPrimaryResearchArticle', 'ScientificPrimaryResearchPreprint'])

## Analyze Collections

In [ ]:
q = ldb.session.query(SKC.id, SKC.name, SKE.id, SKI.type) \
        .filter(SKC.id==SKC_HM.ScientificKnowledgeCollection_id) \
        .filter(SKC_HM.has_members_id==SKE.id) \
        .filter(SKE.id==SKE_HR.ScientificKnowledgeExpression_id) \
        .filter(SKE_HR.has_representation_id==SKI.id) 
df = pd.DataFrame(q.all(), columns=['id', 'collection name', 'doi', 'item type'])
df.pivot_table(index=['id', 'collection name'], columns='item type', values='doi', aggfunc=lambda x: len(x.unique())).fillna(0)   

### Survey + Run Classifications over Papers

In [ ]:
# USE WITH CAUTION - this will delete all extracted metadata notes in the database
# clear all notes across papers listed in `dois` list
l = []
q = ldb.session.query(N, SKE) \
        .filter(N.id == NIA.Note_id) \
        .filter(NIA.is_about_id == SKE.id) \
        .filter(N.type == 'TiAbClassificationNote__cryoet_study_types') \

output = []        
print(len(q.all()))
for n, ske in q.all():
    ldb.delete_note(n.id)    
print(len(q.all()))
    

In [ ]:
t = [t for t in cb.tk.get_tools() if isinstance(t, TitleAbstractClassifier_OneDocAtATime_Tool)][0]
t.run({'collection_id': '5', 'classification_type':'cryoet_study_types', 'repeat_run':True})

In [ ]:
t = [t for t in cb.tk.get_tools() if isinstance(t, TitleAbstractClassifier_OneDocAtATime_Tool)][0]
t.run({'collection_id': '2', 'classification_type':'cryoet_study_types'})


In [ ]:
l = []
q = ldb.session.query(N, SKE) \
        .filter(N.id == NIA.Note_id) \
        .filter(NIA.is_about_id == SKE.id) \
        .filter(N.type == 'TiAbClassificationNote__cryoet_study_types') \

output = []        
for n, ske in q.all():
        tup = json.loads(n.content)
        tup['doi'] = 'http://doi.org/'+re.sub('doi:', '', ske.id)
        tup['year'] = ske.publication_date.year
        tup['month'] = ske.publication_date.month
        tup['ref'] = ske.content
        output.append(tup)
df = pd.DataFrame(output).sort_values(['year', 'month'], ascending=[False, False])
df.to_csv(loc+'/'+db_name+'/cryoet_study_types.tsv', sep='\t')
df


### Survey + Run Extractions over Papers

In [ ]:
t = [t for t in cb.tk.get_tools() if isinstance(t, TitleAbstractExtraction_OneDocAtATime_Tool)][0]
t.run({'collection_id': '5', 'extraction_type':'cryoet'})

## Tests + Checks 


### Agent tool selection + execution + interpretation

In [ ]:
cb.agent_executor.invoke({'input':'Hi who are you and what can you do?'})


## Run MetaData Extraction Chain over listed papers

Here, we run various versions of the metadata extraction tool to examine performance over the cryoet dataset. 

In [ ]:
q = ldb.session.query(SKE.id) \
        .filter(SKC.id==SKC_HM.ScientificKnowledgeCollection_id) \
        .filter(SKC_HM.has_members_id==SKE.id) \
        .filter(SKC.id=='2')  
dois = [e.id for e in q.all()]
dois


In [ ]:
t2 = [t for t in test_tk.get_tools() if isinstance(t, MetadataExtraction_MethodsSectionOnly_Tool)][0]
metadata_dir = '/Users/gully.burns/alhazen/em_tech/empiar/'
t2.compile_answers('cryoet', metadata_dir)
t2.write_answers_as_notes('cryoet', metadata_dir)
#sorted(list(set([doi for q in t2.examples for doi in t2.examples[q]])))

In [ ]:
import local_resources.data_files.cryoet_portal_metadata as cryoet_portal_metadata

# Get the metadata extraction tool
t2 = [t for t in test_tk.get_tools() if isinstance(t, MetadataExtraction_MethodsSectionOnly_Tool)][0]

# Hack to get the path to the metadata directory as a string
#metadata_dir = str(files(cryoet_portal_metadata).joinpath('temp'))[0:-4]
metadata_dir = '/Users/gully.burns/alhazen/em_tech/empiar/'

# Compile the answers from the metadata directory
t2.compile_answers('cryoet', metadata_dir)

# Create a dataframe to store previously extracted metadata
#for d in [d for d_id in dois_to_include for d in dois_to_include[d_id]]:
df = pd.DataFrame()
for d in [d for d in dois]:
    item_types = set()
    l = t2.read_metadata_extraction_notes(d, 'cryoet', 'test')
    df = pd.concat([df, pd.DataFrame(l)]) 
     
# Iterate over papers to run the metadata extraction tool
#for d in [d for d_id in dois_to_include for d in dois_to_include[d_id]]:
for d in [d for d in dois]:
    item_types = set()

    # Skip if the doi is already in the database
    if len(df)>0 and d in df.doi.unique():
        continue

    # Run the metadata extraction tool on the doi
    t2.run(tool_input={'paper_id': d, 'extraction_type': 'cryoet', 'run_label': 'test'})

    # Add the results to the dataframe    
    l2 = t2.read_metadata_extraction_notes(d, 'cryoet', 'test')
    df = pd.concat([df, pd.DataFrame(l2)]) 

In [ ]:

# Create a dataframe to store previously extracted metadata
#for d in [d for d_id in dois_to_include for d in dois_to_include[d_id]]:
t2 = [t for t in test_tk.get_tools() if isinstance(t, MetadataExtraction_MethodsSectionOnly_Tool)][0]
df_final = pd.DataFrame()
for d in [d for d in dois]:
    item_types = set()
    l = t2.read_metadata_extraction_notes(d, 'cryoet', 'test')
    df_final = pd.concat([df_final, pd.DataFrame(l)]) 
df_final

In [ ]:
import local_resources.data_files.cryoet_portal_metadata as cryoet_portal_metadata
from rapidfuzz import fuzz
# Get the metadata extraction tool
t2 = [t for t in test_tk.get_tools() if isinstance(t, MetadataExtraction_MethodsSectionOnly_Tool)][0]

#for d in [d for d_id in dois_to_include for d in dois_to_include[d_id]]:
l = []
for d in [d for d in dois]:
    item_types = set()
    pred1 = t2.read_metadata_extraction_notes(d, 'cryoet', 'test')
    pred2 = t2.read_metadata_extraction_notes(d, 'cryoet', 'test_dbrx')
    gold = t2.read_metadata_extraction_notes(d, 'cryoet', 'gold') 
    if pred1 is None or pred2 is None or gold is None or \
            len(pred1)==0 or len(pred2)==0 or len(gold)!=1:
        continue
    for k in gold[0]:
        g_case = gold[0][k]
        if g_case=='' or g_case is None:
            continue    
        for j, p_case in enumerate(pred1):
            sim = fuzz.ratio(str(g_case), str(p_case.get(k,''))) / 100.0
            print(k, str(g_case), str(p_case.get(k,'')), sim)

In [ ]:
import local_resources.data_files.cryoet_portal_metadata as cryoet_portal_metadata
from rapidfuzz import fuzz
# Get the metadata extraction tool
t2 = [t for t in test_tk.get_tools() if isinstance(t, MetadataExtraction_MethodsSectionOnly_Tool)][0]

df = t2.report_metadata_extraction_for_collection('5', 'cryoet', 'test').set_index('doi')
df.to_csv(loc+'/'+db_name+'/reports/cryoet_metadata_gpt4.tsv', sep='\t')

In [ ]:
ldb.create_zip_archive_of_full_text_files('5', loc+'/'+db_name+'/full_text_files.zip')

In [ ]:
q3 = ldb.session.query(SKE.id, N.name, N.provenance, N.content) \
        .filter(N.id == NIA.Note_id) \
        .filter(NIA.is_about_id == SKE.id) \
        .filter(N.type == 'MetadataExtractionNote') 
l = []
for row in q3.all():
    paper = row[0]
    name = row[1]
#    provenance = json.loads(row[2])
    result = json.loads(row[3])
    kv = {k:result[k] for k in result}
    kv['DOI'] = paper
    kv['run'] = name
    l.append(kv)
# create a dataframe from the list of dictionaries with DOI as the index column
if len(l)>0:
    df = pd.DataFrame(l).set_index(['DOI', 'run'])
else: 
    df = pd.DataFrame()
df 

In [ ]:
# USE WITH CAUTION - this will delete all extracted metadata notes in the database
# clear all notes across papers listed in `dois` list
for row in q3.all():
    d_id = row[0]
    e = ldb.session.query(SKE).filter(SKE.id==d_id).first()
    notes_to_delete = []
    for n in ldb.read_notes_about_x(e):
        notes_to_delete.append(n.id)
    for n in notes_to_delete:
        ldb.delete_note(n)

## Protocol Modeling + Extraction

In [ ]:
ldb = Ceifns_LiteratureDb(loc=loc, name=db_name)
slm = ChatOllama(model='stablelm-zephyr') 
llm = ChatOllama(model='mixtral:instruct') 
llm2 = ChatOpenAI(model='gpt-4-1106-preview') 
llm3 = ChatOpenAI(model='gpt-3.5-turbo') 
d = ("This tool attempts to draw a protocol design from the description of a scientific paper.")

In [ ]:
t1 = ProcotolEntitiesExtractionTool(db=ldb, llm=llm3, description=d)
entities = t1.run(tool_input={'paper_id': 'doi:10.1101/2022.04.12.488077'})
entities

In [ ]:
t2 = ProcotolProcessesExtractionTool(db=ldb, llm=llm3, description=d)
processes = t2.run(tool_input={'paper_id': 'doi:10.1101/2022.04.12.488077'})
processes.get('data')